# k-Nearest Neighbor (kNN) exercise


Классификатор ближайшего соседа работает следующим образом:

- Во время обучения просто запоминает обучающие данные
- Во время тестирования берёт изображение и сравнивает его со всеми обучающими данными, чтобы найти наиболее похожие
- Значение k можно подобрать с помощью кросс-валидации


In [ ]:
# Запустите эту ячейку для проверки работоспособности необходимых модулей.

import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# Немного магии, чтобы заставить графики matplotlib появляться прямо внутри блокнота,
# а не в новом окне.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # размер графиков по умолчанию
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Больше магии с дополнительными Python-модулями можно найти здесь
# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Загрузка исходных данных CIFAR-10.
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

# Очищаем переменные, чтобы данные можно было загружать много раз 
try:
   del X_train, y_train
   del X_test, y_test
   print('Очистка предыдущих данных.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# Выводим размер тестовых и обучающих данных
print('Объём обучающей выборки: ', X_train.shape)
print('Число обучающих меток: ', y_train.shape)
print('Объём тестовой выборки: ', X_test.shape)
print('Число тестовых меток: ', y_test.shape)

In [ ]:
# Отображаем несколько примеров из тестового датасета.
# Посмотрим на несколько изображений из каждого класса.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Делаем дополнительные подвыборки для повышения эффективности
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

# Переформировываем данные в строки
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

In [ ]:
from cs231n.classifiers import KNearestNeighbor

# Создаём классификатор kNN. 
classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

Классифицируем тестовые данные в два этапа:

1. Сначала посчитаем расстояния между всеми тестовыми и обучающими примерами. 
2. Зная эти расстояния, посчитаем k ближайших соседей, которые будут голосовать за метку.

Вычислим матрицу расстояний. Например, если у нас **Ntr** обучающих примеров и **Nte** тестовых, на этом шаге мы должны получить **Nte x Ntr** матрицу, где каждый элемент (i,j) - это расстояние между i-м тыстовым и j-м обучающим примерами.

Для начала откройте файл `cs231n/classifiers/k_nearest_neighbor.py` и реализуйте функцию `compute_distances_two_loops`, которая использует (очень неэффективный) двойной цикл по всем парам примеров (тестовым и обучающим) и вычисляет матрицу расстояний по одному элементу за раз.

In [ ]:
# Откройте файл cs231n/classifiers/k_nearest_neighbor.py и реализуйте функцию
# compute_distances_two_loops.

# Протестируйте вашу функцию:
dists = classifier.compute_distances_two_loops(X_test)
print(dists.shape)

In [ ]:
# Мы можем отобразить матрицу расстояний: каждая строка - это один тестовый пример
# и его расстояния до обучающих примеров
plt.imshow(dists, interpolation='none')
plt.show()

In [ ]:
# Теперь реализуйте функцию predict_labels и запустите код ниже:
# Мы используем k = 1 (Nearest Neighbor).
y_test_pred = classifier.predict_labels(dists, k=1)

# Вычислите и выведите долю правильно спрогнозированных примеров
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Получили %d / %d верных прогнозов => точность: %f' % (num_correct, num_test, accuracy))

Точность должна быть около`27%`. Теперь попробуйте ещё раз с большим значением `k`, например, `k = 5`:

In [ ]:
y_test_pred = classifier.predict_labels(dists, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Получили %d / %d верных прогнозов => точность: %f' % (num_correct, num_test, accuracy))

Точность должна быть немного выше, чем при `k = 1`.

In [ ]:
# Ускорим вычисление матрицы расстояний, используя частичную векторизацию и один цикл. 
# Реализуйте функцию compute_distances_one_loop и запустите код ниже:
dists_one = classifier.compute_distances_one_loop(X_test)

# Чтобы убедиться в правильности векторизованной реализации, сравним её
# с предыдущим тривиальным решением. Один из простейших способов оценки
# похожести двух матриц - норма Фробениуса. Она представляет собой
# квадратный корень квадрата суммы разностей всех элементов;
# другими словами, просто преобразуем матрицы в 2 вектора и посчитаем
# евклидово расстояние между ними.
difference = np.linalg.norm(dists - dists_one, ord='fro')
print('Разница между двумя циклами и одним: %f' % (difference, ))
if difference < 0.001:
    print('Отлично! Расстояние между матрицами одинаковое')
else:
    print('Ой-ой! Расстояние между матрицами разное')

In [ ]:
# Теперь реализуйте полную векторизацию compute_distances_no_loops,
# не используя циклы, и запустите код:
dists_two = classifier.compute_distances_no_loops(X_test)

# Проверьте, что вычисления верны:
difference = np.linalg.norm(dists - dists_two, ord='fro')
print('Разница между двумя циклами и полной векторизацией: %f' % (difference, ))
if difference < 0.001:
    print('Отлично! Расстояние между матрицами одинаковое')
else:
    print('Ой-ой! Расстояние между матрицами разное')

In [ ]:
# Посчитаем, насколько быстро работает наш классификатор
def time_function(f, *args):
    """
    Принимаем функцию f с аргументами и возвращаем время (в секундах), которое понадобилось на выполнение.
    """
    import time
    tic = time.time()
    f(*args)
    toc = time.time()
    return toc - tic

two_loop_time = time_function(classifier.compute_distances_two_loops, X_test)
print('Версия с двумя циклами выполнилась за %f секунд' % two_loop_time)

one_loop_time = time_function(classifier.compute_distances_one_loop, X_test)
print('Версия с одним циклом выполнилась за %f секунд' % one_loop_time)

no_loop_time = time_function(classifier.compute_distances_no_loops, X_test)
print('Версия без циклов выполнилась за %f секунд' % no_loop_time)

# Вы должны увидеть значительное ускорение с полной векторизацией.

# Обратите внимание: в зависимости от вашего компьютера, 
# вы можете не наблюдать ускорения при переходе от двух циклов к одному, 
# и можете даже замедлиться.

### Кросс-валидация

Мы реализовали k-Nearest Neighbor классификатор, но выбирали k вручную. Теперь попробуем определить наилучшее значение k с помощью кросс-валидации.

In [ ]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []
################################################################################
# TODO:                                                                        #
# Разделите данные на небольшие пакеты. После этого X_train_folds и            #
# y_train_folds должны быть списками (list) длиной num_folds, где              #
# y_train_folds[i] это метка для значений X_train_folds[i].                    #
# Подсказка: Посмотрите на функцию numpy array_split.                          #
################################################################################
# *****START OF YOUR CODE*****

pass

# *****END OF YOUR CODE*****

# Словарь, в котором содержатся значения точности для различных значений k, которые
# мы ищем, когда запускаем кросс-валидацию. После этого k_to_accuracies[k] 
# должен быть списком длиной num_folds с разными значениями точности для каждого k.

k_to_accuracies = {}


################################################################################
# TODO:                                                                        #
# Выполните пакетную кросс-валидацию, чтобы найти наилучшее значение k. Для    #
# каждого возможного k запустите алгоритм k-nearest-neighbor num_folds раз,    #
# где в каждом случае используйте все пакеты для обучения, кроме одного, а     #
# последний пакет - оценочные данные. Сохраните значения точности для всех     #
# пакетов и значений k в словарь k_to_accuracies.                              #
################################################################################
# *****START OF YOUR CODE*****

pass

# *****END OF YOUR CODE*****

# Выведите полученные значения точности
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, точность = %f' % (k, accuracy))

In [ ]:
# Нарисуйте график
for k in k_choices:
    accuracies = k_to_accuracies[k]
    plt.scatter([k] * len(accuracies), accuracies)

# Постройте ошибки, которые соответствуют стандартному отклонению
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
plt.title('Кросс-валидация')
plt.xlabel('k')
plt.ylabel('Точность кросс-валидации')
plt.show()

In [ ]:
# По результатам кросс-валидации выберите наилучшее значение k,   
# переобучите классификатор на ВСЕХ обучающих данных и протестируйте на 
# тестовых данных. Точность на тестовых данных должна превышать 28%.
best_k = 1

classifier = KNearestNeighbor()
classifier.train(X_train, y_train)
y_test_pred = classifier.predict(X_test, k=best_k)

# Вычислите и отобразите точность
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Получили %d / %d верных прогнозов => точность: %f' % (num_correct, num_test, accuracy))